In [0]:
import numpy
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
import numpy as np
import keras


In [6]:
numpy.random.seed(7)
# load data
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
# normalize inputs from 0-255 to 0.0-1.0
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train / 255.0
X_test = X_test / 255.0
# one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

170500096/170498071 [==============================] - 2s 0us/step


In [18]:
num_classes = y_train.shape[1]
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(32, 32, 3), padding='same', activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(0.2))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', kernel_constraint=maxnorm(3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
# Compile model
epochs = 25
lrate = 0.01
decay = lrate/epochs
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
print(model.summary())

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
dropout_15 (Dropout)         (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
flatten_8 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_15 (Dense)             (None, 512)               4194816   
_________________________________________________________________
dropout_16 (Dropout)         (None, 512)              

In [19]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=32)
# Final evaluation of the model

Train on 50000 samples, validate on 10000 samples
Epoch 1/25
50000/50000 [==============================] - 11s 223us/step - loss: 1.6890 - acc: 0.3894 - val_loss: 1.4107 - val_acc: 0.4923
Epoch 2/25
50000/50000 [==============================] - 11s 215us/step - loss: 1.3658 - acc: 0.5058 - val_loss: 1.2337 - val_acc: 0.5664
Epoch 3/25
50000/50000 [==============================] - 11s 215us/step - loss: 1.2041 - acc: 0.5690 - val_loss: 1.1213 - val_acc: 0.6060
Epoch 4/25
50000/50000 [==============================] - 11s 217us/step - loss: 1.0790 - acc: 0.6173 - val_loss: 1.0705 - val_acc: 0.6213
Epoch 5/25
50000/50000 [==============================] - 11s 218us/step - loss: 0.9777 - acc: 0.6523 - val_loss: 1.0239 - val_acc: 0.6363
Epoch 6/25
50000/50000 [==============================] - 11s 216us/step - loss: 0.8912 - acc: 0.6866 - val_loss: 0.9814 - val_acc: 0.6540
Epoch 7/25
50000/50000 [==============================] - 11s 216us/step - loss: 0.8184 - acc: 0.7090 - val_loss: 0.

Next we will be adding the changes that were requested to us in the icp

In [37]:
model2 = Sequential()
model2.add(Conv2D(32, (3, 3), input_shape=(32, 32, 3), padding='same', activation='relu'))
model2.add(Dropout(0.2))
model2.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model2.add(Dropout(0.2))
model2.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model2.add(MaxPooling2D(pool_size=(2,2)))
model2.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model2.add(Dropout(0.2))
model2.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(Flatten())
model2.add(Dropout(0.2))
model2.add(Dense(1024, activation='relu'))
model2.add(Dropout(0.2))
model2.add(Dense(512, activation='relu'))
model2.add(Dropout(0.2))
model2.add(Dense(10, activation='softmax'))
model2.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
print(model2.summary())

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_91 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
dropout_75 (Dropout)         (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_92 (Conv2D)           (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d_45 (MaxPooling (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_93 (Conv2D)           (None, 16, 16, 64)        18496     
_________________________________________________________________
dropout_76 (Dropout)         (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_94 (Conv2D)           (None, 16, 16, 64)      

So it would appear that for whatever reason, the accuracty of the model went down.... Perhaps it may be possible that we may be doing too many convolutions on the dataset?

In [43]:
tbCallBack = keras.callbacks.TensorBoard(log_dir='./Graph', histogram_freq=0,write_graph=True, write_images=True) 
model2.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=32, callbacks=[tbCallBack])
# Final evaluation of the model
scores = model2.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Train on 50000 samples, validate on 10000 samples


Epoch 1/25
50000/50000 [==============================] - 13s 267us/step - loss: 2.2898 - acc: 0.1177 - val_loss: 2.2729 - val_acc: 0.2045

Epoch 2/25
50000/50000 [==============================] - 13s 258us/step - loss: 2.1655 - acc: 0.1942 - val_loss: 2.0420 - val_acc: 0.2679
Epoch 3/25
50000/50000 [==============================] - 13s 260us/step - loss: 2.0077 - acc: 0.2554 - val_loss: 2.0159 - val_acc: 0.2827
Epoch 4/25
50000/50000 [==============================] - 13s 254us/step - loss: 1.9391 - acc: 0.2893 - val_loss: 1.9308 - val_acc: 0.3223
Epoch 5/25
50000/50000 [==============================] - 13s 251us/step - loss: 1.8706 - acc: 0.3192 - val_loss: 2.0171 - val_acc: 0.2906
Epoch 6/25
50000/50000 [==============================] - 13s 253us/step - loss: 1.8022 - acc: 0.3400 - val_loss: 1.8756 - val_acc: 0.3354
Epoch 7/25
50000/50000 [==============================] - 13s 250us/step - loss: 1.7494 - acc: 0.3574 - val_loss:

In [0]:
!tensorboard --logdir ./Graph

Tensorboard does not run on colab so I had to download the Graph directory and run it locally on my machine.

In [65]:
predictions = model2.predict_classes(X_test[:4])
mix = zip(predictions, y_test[:4])
print('\n'.join(["actual: {} predicted: {}".format(z[0], str(np.where(z[1] == 1)[0])) for z in mix]))

actual: 3 predicted: [3]
actual: 1 predicted: [8]
actual: 8 predicted: [8]
actual: 8 predicted: [0]


Looks like, out of the ones we tested against, only half of them were acurate